In [1]:
import os
import sys
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext

In [2]:
conf = SparkConf().setAppName('G101HW1').setMaster("local[*]")
sc = SparkContext(conf=conf)

22/04/06 16:12:10 WARN Utils: Your hostname, eunoia resolves to a loopback address: 127.0.1.1; using 192.168.196.57 instead (on interface wlp1s0)
22/04/06 16:12:10 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/04/06 16:12:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
""" 
    K = <int> Number of Partitions
    H = <int> Number of Products with Highest Popularity
    S = <str> Name of the Country -- 'all' Means all Countries
    dataset_path = Path of the Dataset File

"""
K=4
H=5
# S='United_Kingdom'
# S = 'all'
S = 'Italy'
# dataset_path = 'sample_50.csv'
dataset_path = 'sample_10000.csv'
# dataset_path = 'full_dataset.csv'
# sqlContext = SQLContext(sc)
rawData = sc.textFile(dataset_path,K).cache()
rawData.repartition(K)

MapPartitionsRDD[6] at coalesce at NativeMethodAccessorImpl.java:0

In [4]:
rawData.count()

10000

In [5]:
rawData.getNumPartitions()

4

In [6]:
def productCustomer(row, country='all'):
    """
    row = [0:TransactionID, 1:ProductID, 2:Description, 3:Quantity, 4:InvoiceDate, 5:UnitPrice, 6:CustomerID, 7:Country]
    """
    s = row.split(',')
    if int(s[3]) > 0:
        if country == 'all':
            return ((s[1], int(s[6])),0)
        elif s[7] == country:
            return ((s[1], int(s[6])),0)
        
        
product_customer = (rawData.map(lambda row: productCustomer(row, S)).filter(lambda row: row)
                    .groupByKey()
                    .map(lambda x: x[0]))
print(f'Product-Customer Pairs = {product_customer.count()}')

Product-Customer Pairs = 20


In [8]:
def productCustomer2(row, country='all'):
    """
    row = [0:TransactionID, 1:ProductID, 2:Description, 3:Quantity, 4:InvoiceDate, 5:UnitPrice, 6:CustomerID, 7:Country]
    """
    s = row.split(',')
    if int(s[3]) > 0:
        if country == 'all':
            return [((s[1], int(s[6])),0)]
        elif s[7] == country:
            return [((s[1], int(s[6])),0)]
        else:
            return []
    else:
        return []
    
    
product_customer2 = (rawData.flatMap(lambda row: productCustomer2(row, S))
                    .groupByKey()
                    .map(lambda x: x[0]))
print(f'Product-Customer Pairs = {product_customer.count()}')

Product-Customer Pairs = 20


In [25]:
def changeValueToOne(iterator):
    for pair in iter(iterator):
        yield (pair[0],1)
        
product_popularity1 = (product_customer
                       .mapPartitions(changeValueToOne)
                        .groupByKey()
                        .mapValues(sum)
                      )
product_popularity1.count()

17

In [26]:
product_popularity1.collect()

[('23084', 1),
 ('22556', 1),
 ('22776', 1),
 ('22273', 1),
 ('22844', 1),
 ('22847', 1),
 ('22720', 1),
 ('22961', 1),
 ('21623', 1),
 ('22804', 1),
 ('22423', 2),
 ('84375', 1),
 ('21755', 1),
 ('22845', 1),
 ('84879', 1),
 ('POST', 3),
 ('20718', 1)]

In [28]:
product_popularity2 = (product_customer.map(lambda x:(x[0],1))
                       .reduceByKey(lambda x,y:x+y)
)
product_popularity2.collect()

[('23084', 1),
 ('22556', 1),
 ('22776', 1),
 ('22273', 1),
 ('22844', 1),
 ('22847', 1),
 ('22720', 1),
 ('22961', 1),
 ('21623', 1),
 ('22804', 1),
 ('22423', 2),
 ('84375', 1),
 ('21755', 1),
 ('22845', 1),
 ('84879', 1),
 ('POST', 3),
 ('20718', 1)]

In [35]:
highest_popularity = (product_popularity1
                     .sortBy(lambda x: x[1],ascending=False)
                     ).take(H)
highest_popularity

[('POST', 3), ('22423', 2), ('23084', 1), ('22556', 1), ('22776', 1)]

In [13]:
# def takeHighestPopularity(iterator,H):
#     highest_popularity = list()
#     for i in range(H):
#         highest_popularity.append(iterator[i])
#     return highest_popularity
        

In [ ]:
# highest_popularity= (product_popularity1
#                      .map(lambda x: (x[1],x[0]))
#                      .repartitionAndSortWithinPartitions(4,ascending=True)
# #                      .foreachPartition(lambda l: print(x) for x in l)
#                     )
# highest_popularity.collect()

In [36]:
a=['1','POST','5']
a.sort()
print(a)

['1', '5', 'POST']


In [38]:
sorted_product_popularity1= (product_popularity1
                             .sortByKey()
                            ).collect()
sorted_product_popularity1

[('20718', 1),
 ('21623', 1),
 ('21755', 1),
 ('22273', 1),
 ('22423', 2),
 ('22556', 1),
 ('22720', 1),
 ('22776', 1),
 ('22804', 1),
 ('22844', 1),
 ('22845', 1),
 ('22847', 1),
 ('22961', 1),
 ('23084', 1),
 ('84375', 1),
 ('84879', 1),
 ('POST', 3)]

In [39]:
sorted_product_popularity2= (product_popularity2
                             .sortByKey()
                            ).collect()
sorted_product_popularity2

[('20718', 1),
 ('21623', 1),
 ('21755', 1),
 ('22273', 1),
 ('22423', 2),
 ('22556', 1),
 ('22720', 1),
 ('22776', 1),
 ('22804', 1),
 ('22844', 1),
 ('22845', 1),
 ('22847', 1),
 ('22961', 1),
 ('23084', 1),
 ('84375', 1),
 ('84879', 1),
 ('POST', 3)]